# Resampling Images

## Overview

### Define output image information

* When resampling an image, we must first define output the image information, i.e. the *Origin*, *Spacing*, *Direction*, and *Size*.
* This determines the grid of samples where the output will be sampled.
* It does need not need to be the same as the input image.

![Resampling output information](Data/ResampleOutputInformation.png)

### Finding a resampled pixel's value

For each pixel in the output image:

1. Transform its index value to its location in physical space.
2. Apply an optional spatial transformation.
3. Transform the physical point location to a continuous index on the image to be resampled.
4. Interpolate from neighboring pixels.

![Common reference system](Data/CommonReferenceSystem.png)

### Different interpolators

<a href="http://www.itk.org/SimpleITKDoxygen/html/namespaceitk_1_1simple.html#a7cb1ef8bd02c669c02ea2f9f5aa374e5">There are many interpolators available</a>:
<ul>
<li> sitkNearestNeighbor </li>
<li> sitkLinear </li>
<li> sitkBSpline </li>
<li> sitkGaussian </li>
<li> sitkHammingWindowedSinc </li>
<li> sitkCosineWindowedSinc </li>
<li> sitkWelchWindowedSinc </li>
<li> sitkLanczosWindowedSinc </li>
<li> sitkBlackmanWindowedSinc </li>
</ul>

### Resampling and the registration framework

**Registration** involves **sampling** the two images to be registered with at **spatial transform**. The sampled values are compared with a **similarity metric**.  An **optimizer** is used to improve the spatial transformation parameters iteratively until the samples coorespond.

![Registration framework](Data/ITKv4RegistrationComponentsDiagram.svg)

## Tutorial

In [ ]:
import SimpleITK as sitk

from matplotlib import pyplot as plt
%matplotlib inline

import numpy as np

Read a 3D input image.

In [ ]:
input_image = sitk.ReadImage('Data/NESb_C2_TP1.tiff')
print(input_image.GetSize())

View one of its slices.

In [ ]:
input_slice = input_image[:,:,4]
plt.imshow(sitk.GetArrayFromImage(input_slice), cmap='gray')

Create a resampler.

In [ ]:
resampler = sitk.ResampleImageFilter()

What are the options that can be set on the `resampler`?

In [ ]:
# Uncomment the line below and place the cursor after the "t" in "Set"
# Then, press the TAB key.
#resampler.Set

We can either set use a *reference image* to determine the output image pixel sampling grid, or we can set desired output image information manually.

Here we will make the output image size half of the input image size in each direction.

In [ ]:
origin = input_image.GetOrigin()
resampler.SetOutputOrigin(origin)

direction = input_image.GetDirection()
resampler.SetOutputDirection(direction)

spacing = input_image.GetSpacing()
resampler.SetOutputSpacing(spacing)

size = list(input_image.GetSize())
size[0] = int(size[0] / 2)
size[1] = int(size[1] / 2)
resampler.SetSize(size)
print(size)

In [ ]:
resampled_image = resampler.Execute(input_image)
plt.imshow(sitk.GetArrayFromImage(resampled_image[:,:,4]), cmap='gray')

We can also apply a spatial transform in the resampling process.

In [ ]:
dimension = 3
translation = (-100.0, -200.0, 0.0)
transform = sitk.TranslationTransform(dimension, translation)
resampler.SetTransform(transform)

In [ ]:
resampled_image = resampler.Execute(input_image)
plt.imshow(sitk.GetArrayFromImage(resampled_image[:,:,4]), cmap='gray')

When the input image is sampled outside of its domain, a *default pixel value* is returned.

In [ ]:
resampler.SetDefaultPixelValue(150)

In [ ]:
resampled_image = resampler.Execute(input_image)
plt.imshow(sitk.GetArrayFromImage(resampled_image[:,:,4]), cmap='gray')

Finally, we can also change the interpolator used.  The default is a bi-linear interpolator for 2D images and a tri-linear interpolator for 3D images.

In [ ]:
resampler.SetInterpolator(sitk.sitkNearestNeighbor)

In [ ]:
resampled_image = resampler.Execute(input_image)
plt.imshow(sitk.GetArrayFromImage(resampled_image[:,:,4]), cmap='gray')

## Exercises

### Exercise 1: Windowed Sinc

Run the `resampler` with the `sitk.sitkWelchWindowedSinc` interpolator.  What is different?

### Exercise 2: Interpolator behavior

The behavior of different interpolators is demonstrated on the <a href="http://www.cs.cornell.edu/~srm/publications/Vis94-filters-abstract.html">Marschner-Lobb</a> image with the code below.

Which interpolators before best? Is performance related to the experiences from Exercise 1?

In [ ]:
import math

def marschner_lobb(size=40, alpha=0.25, f_M=6.0):
    img = sitk.PhysicalPointSource(sitk.sitkVectorFloat32, [size]*3, [-1]*3, [2.0/size]*3)
    imgx = sitk.VectorIndexSelectionCast(img, 0)
    imgy = sitk.VectorIndexSelectionCast(img, 1)
    imgz = sitk.VectorIndexSelectionCast(img, 2)
    del img
    r = sitk.Sqrt(imgx**2 + imgy**2)
    del imgx, imgy
    pr = sitk.Cos((2.0*math.pi*f_M)*sitk.Cos((math.pi/2.0)*r))
    ml = (1.0 - sitk.Sin((math.pi/2.0)*imgz) + alpha*(1.0+pr))/(2.0*(1.0+alpha))
    return ml[:,:,ml.GetSize()[-1]//2]

def myshow(image, title):
    arr = sitk.GetArrayFromImage(image)
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(arr, cmap='gray')
    ax.set_title(title)
    
def expand(image, interpolator):
    return sitk.Expand(image, [5]*2, interpolator)

interpolators = [('Nearest Neighbor', sitk.sitkNearestNeighbor),
                 ('Linear', sitk.sitkLinear),
                 ('B-Spline', sitk.sitkBSpline),
                 ('Gaussian', sitk.sitkGaussian),
                 ('Hamming Windowed Sinc', sitk.sitkHammingWindowedSinc),
                 ('Blackman Windowed Sinc', sitk.sitkBlackmanWindowedSinc),
                 ('Cosine Windowed Sinc', sitk.sitkCosineWindowedSinc),
                 ('Welch Windowed Sinc', sitk.sitkWelchWindowedSinc),
                 ('Lanczos Windowed Sinc', sitk.sitkLanczosWindowedSinc)]

image = marschner_lobb()
for interpolator in interpolators:
    expanded = expand(image, interpolator[1])
    myshow(expanded, interpolator[0])
    